In [1]:
import numpy as np
import pandas as pd

__Описание задачи__

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать recall@k и precision@k.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

__Входные данные__

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через `,` затем идёт `;` после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, `1,2,3,4;` или `1,2,3,4;5,6`.

Гарантируется, что среди id купленных товаров все различные.

__Важно:__

Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
Если товар не встречался в обучающей выборке, его популярность равна 0.
Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.

__Задание__

1. На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)
1. Реализуйте два алгоритма рекомендаций:
    - сортировка просмотренных id по популярности (частота появления в просмотренных),
    - сортировка просмотренных id по покупаемости (частота появления в покупках).
3. Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

__Дополнительные вопросы__

Обратите внимание, что при сортировке по покупаемости возникает много товаров с одинаковым рангом - это означает, что значение метрик будет зависеть от того, как мы будем сортировать товары с одинаковым рангом. Попробуйте убедиться, что при изменении сортировки таких товаров recall@k меняется. Подумайте, как оценить минимальное и максимальное значение recall@k в зависимости от правила сортировки.
Мы обучаемся и тестируемся на полных сессиях (в которых есть все просмотренные за сессию товары). Подумайте, почему полученная нами оценка качества рекомендаций в этом случае несколько завышена.

In [2]:
df = pd.read_csv('data/coursera_sessions_train.txt', sep=';', header=None)
df.columns = ['viewed', 'bought']
df.index = range(len(df))
df.viewed = df.viewed.apply(lambda x: list(map(int, x.split(','))))
df.bought = df.bought.dropna().apply(lambda x: list(map(int, x.split(','))))
df.head(10)

,viewed,bought
0,"[0, 1, 2, 3, 4, 5]",NaN
1,"[9, 10, 11, 9, 11, 12, 9, 11]",NaN
2,"[16, 17, 18, 19, 20, 21]",NaN
3,"[24, 25, 26, 27, 24]",NaN
4,"[34, 35, 36, 34, 37, 35, 36, 37, 38, 39, 38, 39]",NaN
5,[42],NaN
6,"[47, 48, 49]",NaN
7,"[59, 60, 61, 62, 60, 63, 64, 65, 66, 61, 67, 6...","[67, 60, 63]"
8,"[71, 72, 73, 74]",NaN
9,"[76, 77, 78]",NaN


In [3]:
from collections import Counter, OrderedDict
class OrderedCounter(Counter, OrderedDict):
    pass

In [7]:
viewed = np.concatenate(df.viewed)
bought = np.concatenate(df.bought.dropna().values)

top_viewed = OrderedDict(
    sorted(Counter(viewed).items(),
           key=lambda x: x[1],
           reverse=True))

top_bought = OrderedDict(
    sorted(Counter(bought).items(),
           key=lambda x: x[1],
           reverse=True))

In [8]:
top_viewed

OrderedDict([(73, 677),
             (158, 641),
             (204, 396),
             (262, 387),
             (162, 318),
             (7, 312),
             (137, 306),
             (1185, 284),
             (6, 283),
             (170, 280),
             (800, 253),
             (5202, 227),
             (8, 225),
             (609, 213),
             (3149, 213),
             (3324, 204),
             (1346, 199),
             (751, 197),
             (1184, 186),
             (1933, 186),
             (1283, 186),
             (1844, 186),
             (325, 184),
             (551, 177),
             (4604, 177),
             (1334, 174),
             (42149, 174),
             (259, 173),
             (1852, 172),
             (70238, 170),
             (72, 167),
             (258, 167),
             (758, 166),
             (85, 165),
             (1342, 165),
             (343, 164),
             (2290, 163),
             (5501, 160),
             (1323, 159),
             (

#### Алгоритм рекомендаций на основе сортировки просмотренных по популярности среди всех

Этот алгоритм работает со списком запросов пользователя. Он выбрасывает дублирующиеся запросы из списка, затем для каждого запроса извлекает значение количества просмотров/покупок из топов  `top_viewed`, таким образом присваивая объективную популярность запросу, затем, он сортирует список запросов по популярности. Если популярность одинаковая, то сортировка происходит по позврастанию момента запроса (чем раньше пользователь запросил id, тем выше он в рекомендациях)

In [9]:
df.dropna().head()

,viewed,bought
7,"[59, 60, 61, 62, 60, 63, 64, 65, 66, 61, 67, 6...","[67, 60, 63]"
10,"[84, 85, 86, 87, 88, 89, 84, 90, 91, 92, 93, 86]",[86]
19,"[138, 198, 199, 127]",[199]
30,"[303, 304, 305, 306, 307, 308, 309, 310, 311, ...",[303]
33,"[352, 353, 352]",[352]


In [14]:
def query_pop_sort(query):
    # Drops duplicated query items with saving the order
    oc = OrderedCounter(query).keys()
    # Gets views total count for every item in query
    poplist = []
    for x in list(oc):
        if x in top_viewed:
            poplist.append(top_viewed[x])
        else:
            poplist.append(0)
    # poplist = [top_viewed[x] for x in list(oc) if x in top_viewed]
    # Make timestamps for query
    l = list(zip(oc, poplist, range(len(oc))))
    # Sorted list represents query sorted by total view counts, then by timestamp
    sorted_list = sorted(
        l, key=lambda x: (x[1], -x[2]), reverse=True)  # '-' for ascending timestamp
    sorted_list = [x[0] for x in sorted_list]
    return sorted_list


In [15]:
query_pop_sort(df.viewed[7])


[63, 64, 60, 61, 65, 66, 67, 68, 59, 62]

#### Алгоритм рекомендаций на основе сортировки просмотренных по покупаемости среди всех

Аналогично предыдущему алгоритм сортирует запросы пользователя по покупаемости товара.

In [28]:
def query_bought_sort(query):
    # Drops duplicated query items with saving the order
    oc = OrderedCounter(query).keys()  
    # Gets views total count for every item in query
    boughtlist = []
    for x in list(oc):
        if x in top_bought:
            boughtlist.append(top_bought[x])
        else:
            boughtlist.append(0)
    # boughtlist = [top_bought[x] for x in list(oc) if x in top_bought]  
    # Zips id, total bought count for this id and timestamps of ids in query
    l = list(zip(oc, boughtlist, range(len(oc))))
    # Sorted list represents query sorted by total buy counts, then by timestamp
    sorted_list = sorted(
        l, key=lambda x: (x[1], -x[2]), reverse=True)  # '-' for ascending timestamp
    sorted_list = [x[0] for x in sorted_list]
    return sorted_list

In [29]:
query_bought_sort(df.viewed[7])


[60, 63, 67, 59, 61, 62, 64, 65, 66, 68]

Оба алгоритма формировали популярность и покупаемость `top_views`, `top_boughts` по обучающей выборке.

Сформируем рекомендации для обучающей выборки, затем для контрольной выборки.

In [30]:
df['rec_by_pop'] = (df.viewed.apply(query_pop_sort))
df['rec_by_bought'] = (df.viewed.apply(query_bought_sort))
df = df.dropna()
df.head()

,viewed,bought,rec_by_pop,rec_by_bought
7,"[59, 60, 61, 62, 60, 63, 64, 65, 66, 61, 67, 6...","[67, 60, 63]","[63, 64, 60, 61, 65, 66, 67, 68, 59, 62]","[60, 63, 67, 59, 61, 62, 64, 65, 66, 68]"
10,"[84, 85, 86, 87, 88, 89, 84, 90, 91, 92, 93, 86]",[86],"[85, 93, 89, 90, 84, 92, 86, 87, 91, 88]","[86, 85, 93, 84, 87, 88, 89, 90, 91, 92]"
19,"[138, 198, 199, 127]",[199],"[127, 138, 198, 199]","[138, 199, 127, 198]"
30,"[303, 304, 305, 306, 307, 308, 309, 310, 311, ...",[303],"[303, 306, 304, 307, 309, 310, 305, 308, 311, ...","[303, 304, 305, 306, 307, 308, 309, 310, 311, ..."
33,"[352, 353, 352]",[352],"[352, 353]","[352, 353]"


In [31]:
df_test = pd.read_csv('data/coursera_sessions_test.txt', sep=';', header=None).dropna()
df_test.columns = ['viewed', 'bought']
df_test.index = range(len(df_test))
df_test = df_test.applymap(lambda x: list(map(int, x.split(','))))

df_test['rec_by_pop'] = (df_test.viewed.apply(query_pop_sort))
df_test['rec_by_bought'] = (df_test.viewed.apply(query_bought_sort))

df_test.head()


,viewed,bought,rec_by_pop,rec_by_bought
0,"[63, 68, 69, 70, 66, 61, 59, 61, 66, 68]","[66, 63]","[63, 68, 66, 61, 59, 69, 70]","[63, 68, 69, 70, 66, 61, 59]"
1,"[158, 159, 160, 159, 161, 162]",[162],"[158, 162, 160, 159, 161]","[158, 162, 160, 159, 161]"
2,"[200, 201, 202, 203, 204]","[201, 205]","[204, 202, 203, 200, 201]","[204, 202, 200, 201, 203]"
3,"[371, 372, 371]","[371, 373]","[371, 372]","[371, 372]"
4,[422],[422],[422],[422]


In [32]:
def pk(actual, predicted, k=10):
    if len(predicted)>k:
        predicted = predicted[:k]
    return sum([x in actual for x in predicted]) / k

def avg_pk(actual, predicted, k=10):
    return np.mean([pk(a,p,k) for a,p in zip(actual, predicted)])

In [33]:
def rk(actual, predicted, k=10):
    if len(predicted)>k:
        predicted = predicted[:k]
    return sum([x in actual for x in predicted]) / len(actual)

def avg_rk(actual, predicted, k=10):
    return np.mean([rk(a,p,k) for a,p in zip(actual, predicted)])

In [34]:
df.head()

,viewed,bought,rec_by_pop,rec_by_bought
7,"[59, 60, 61, 62, 60, 63, 64, 65, 66, 61, 67, 6...","[67, 60, 63]","[63, 64, 60, 61, 65, 66, 67, 68, 59, 62]","[60, 63, 67, 59, 61, 62, 64, 65, 66, 68]"
10,"[84, 85, 86, 87, 88, 89, 84, 90, 91, 92, 93, 86]",[86],"[85, 93, 89, 90, 84, 92, 86, 87, 91, 88]","[86, 85, 93, 84, 87, 88, 89, 90, 91, 92]"
19,"[138, 198, 199, 127]",[199],"[127, 138, 198, 199]","[138, 199, 127, 198]"
30,"[303, 304, 305, 306, 307, 308, 309, 310, 311, ...",[303],"[303, 306, 304, 307, 309, 310, 305, 308, 311, ...","[303, 304, 305, 306, 307, 308, 309, 310, 311, ..."
33,"[352, 353, 352]",[352],"[352, 353]","[352, 353]"


In [35]:
query_pop_sort(df.viewed[7])

[63, 64, 60, 61, 65, 66, 67, 68, 59, 62]

In [36]:
df.bought[7]

[67, 60, 63]

In [37]:
pk(df.bought[7], df.rec_by_pop[7], k=5)

0.4

In [79]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
        A list of elements that are to be predicted (order doesn't matter)
    predicted : list
        A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

In [80]:
def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [81]:
def ark(actual, predicted, k=10):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / len(actual)

In [82]:
def mark(actual, predicted, k=10):
    return np.mean([ark(a,p,k) for a,p in zip(actual, predicted)])

In [83]:
mark(df.bought, df.rec_by_pop, k=5)

0.6145435213371051

## Train sample

### Popularity algo:

In [38]:
pop_algo_train = []

**AverageRecall@1**

In [39]:
ans = avg_rk(df.bought.values, df.rec_by_pop.values, k=1)
pop_algo_train.append(round(ans, 2))

**AveragePrecision@1**

In [40]:
ans = avg_pk(df.bought.values, df.rec_by_pop.values, k=1)
pop_algo_train.append(round(ans, 2))

**AverageRecall@5**

In [41]:
ans = avg_rk(df.bought.values, df.rec_by_pop.values, k=5)
pop_algo_train.append(round(ans, 2))

**AveragePrecision@5**

In [42]:
ans = avg_pk(df.bought.values, df.rec_by_pop.values, k=5)
pop_algo_train.append(round(ans, 2))

In [43]:
pop_algo_train

[0.44, 0.51, 0.82, 0.21]

In [44]:
with open('pop_algo_train.txt', 'w') as fout:
    fout.write(' '.join([str(x) for x in pop_algo_train]))

!cat pop_algo_train.txt

0.44 0.51 0.82 0.21

### Bought algo

**AverageRecall@1**

In [45]:
bought_algo_train = []

In [46]:
ans = avg_rk(df.bought.values, df.rec_by_bought.values, k=1)
bought_algo_train.append(round(ans, 2))

**AveragePrecision@1**

In [47]:
ans = avg_pk(df.bought.values, df.rec_by_bought.values, k=1)
bought_algo_train.append(round(ans, 2))

**AverageRecall@5**

In [48]:
ans = avg_rk(df.bought.values, df.rec_by_bought.values, k=5)
bought_algo_train.append(round(ans, 2))

**AveragePrecision@5**

In [49]:
ans = avg_pk(df.bought.values, df.rec_by_bought.values, k=5)
bought_algo_train.append(round(ans, 2))

In [50]:
bought_algo_train

[0.69, 0.8, 0.93, 0.25]

In [51]:
with open('bought_algo_train.txt', 'w') as fout:
    fout.write(' '.join([str(x) for x in bought_algo_train]))

!cat bought_algo_train.txt

0.69 0.8 0.93 0.25

## Test sample

### Popularity algo:

In [52]:
pop_algo_test = []

**AverageRecall@1**

In [53]:
ans = avg_rk(df_test.bought.values, df_test.rec_by_pop.values, k=1)
pop_algo_test.append(round(ans, 2))

**AveragePrecision@1**

In [54]:
ans = avg_pk(df_test.bought.values, df_test.rec_by_pop.values, k=1)
pop_algo_test.append(round(ans, 2))

**AverageRecall@5**

In [55]:
ans = avg_rk(df_test.bought.values, df_test.rec_by_pop.values, k=5)
pop_algo_test.append(round(ans, 2))

**AveragePrecision@5**

In [56]:
ans = avg_pk(df_test.bought.values, df_test.rec_by_pop.values, k=5)
pop_algo_test.append(round(ans, 2))

In [57]:
pop_algo_test

[0.42, 0.48, 0.8, 0.2]

In [58]:
with open('pop_algo_test.txt', 'w') as fout:
    fout.write(' '.join([str(x) for x in pop_algo_test]))

!cat pop_algo_test.txt

0.42 0.48 0.8 0.2

### Bought algo

In [59]:
bought_algo_test = []

**AverageRecall@1**

In [60]:
ans = avg_rk(df_test.bought.values, df_test.rec_by_bought.values, k=1)
bought_algo_test.append(round(ans, 2))

**AveragePrecision@1**

In [61]:
ans = avg_pk(df_test.bought.values, df_test.rec_by_bought.values, k=1)
bought_algo_test.append(round(ans, 2))

**AverageRecall@5**

In [62]:
ans = avg_rk(df_test.bought.values, df_test.rec_by_bought.values, k=5)
bought_algo_test.append(round(ans, 2))

**AveragePrecision@5**

In [63]:
ans = avg_pk(df_test.bought.values, df_test.rec_by_bought.values, k=5)
bought_algo_test.append(round(ans, 2))

In [64]:
bought_algo_test

[0.46, 0.53, 0.82, 0.21]

In [65]:
with open('bought_algo_test.txt', 'w') as fout:
    fout.write(' '.join([str(x) for x in bought_algo_test]))

!cat bought_algo_test.txt

0.46 0.53 0.82 0.21